In [ ]:
import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

from datasets import load_dataset
from transformer_lens.utils import tokenize_and_concatenate
from sae_lens import SAE

import tools
import latent_features

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small", fold_ln=True)

layer_index = 6
location = "mlp_post_act"
transformer_lens_loc = f"blocks.{layer_index}.mlp.hook_post"
prev_layer_loc = f"blocks.{layer_index}.ln2.hook_normalized"

In [ ]:
ds = load_dataset("NeelNanda/pile-10k", split='train[:10]')
ds_tokens = model.to_tokens(ds['text'])
ds_logits, ds_cache = model.run_with_cache(ds_tokens[0])

In [ ]:
mlp = tools.extract_mlp(model, layer_index)

ds_acts = ds_cache[prev_layer_loc].numpy().reshape(-1, 768)
original_mlp = tools.get_original_mlp_for_sparx(mlp, ds_acts)

shrink_pcs = [0.1, 0.3, 0.5, 0.7, 0.9]
merged_models = []
models_cluster_labels = []
for pc in shrink_pcs:
  model, labels = tools.shrink_model_global(original_mlp, pc)
  model.model.summary()
  merged_models.append(model)
  models_cluster_labels.append(labels)